In [3]:
import torch
import torch.nn as nn

In [5]:
rnn = nn.RNN(input_size=10, hidden_size=20, num_layers=2)

In [6]:
print("wih形状{},whh形状{},bih形状{}".format(rnn.weight_ih_l0.shape,
                                       rnn.weight_hh_l0.shape,
                                       rnn.bias_hh_l0.shape))

wih形状torch.Size([20, 10]),whh形状torch.Size([20, 20]),bih形状torch.Size([20])


In [7]:
print("wih形状{},whh形状{},bih形状{}".format(rnn.weight_ih_l1.shape,
                                       rnn.weight_hh_l1.shape,
                                       rnn.bias_hh_l1.shape))

wih形状torch.Size([20, 20]),whh形状torch.Size([20, 20]),bih形状torch.Size([20])


In [9]:
input=torch.randn(100,32,10)
h_0=torch.randn(2,32,20)

In [10]:
output,h_n=rnn(input,h_0)
print(output.shape,h_n.shape)

torch.Size([100, 32, 20]) torch.Size([2, 32, 20])


## 单元版RNN实现

In [ ]:
rnn_cell = nn.RNNCell(input_size=10, hidden_size=20)

input_cell=torch.randn(100,32,10)

h_cell_0=torch.randn(32,20)
output = []

In [ ]:
# 手动实现RNN

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
#rnn = RNN(n_letters, n_hidden, n_categories)

In [4]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM以word_embeddings作为输入，输出维度为hidden_dim的隐藏状态值
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # 线性层将隐藏状态空间映射到标注空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 一开始并没有隐藏状态所以要先初始化一个
        #(num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1,1,self.hidden_dim),
                torch.zeros(1,1,self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores